In [1]:
%cd ~/Dev/video-membership

/Users/cfe/Dev/video-membership


In [2]:
from app import db
from app.users.models import User
from cassandra.cqlengine.management import sync_table

In [3]:
db.get_session()
sync_table(User)

In [21]:
post_data = {
    "email": "abc@abc.com",
    "password": "abc123"
}

In [25]:
from pydantic import BaseModel, EmailStr, SecretStr, validator

In [23]:
class UserLoginSchema(BaseModel):
    email: EmailStr
    password: SecretStr

In [24]:
UserLoginSchema(**post_data) # email="abc@abc", password="abc123"

UserLoginSchema(email='abc@abc.com', password=SecretStr('**********'))

In [39]:
class UserSignupSchema(BaseModel):
    email: EmailStr
    password: SecretStr
    password_confirm: SecretStr
        
    
    @validator("email")
    def email_available(cls, v, values, **kwargs):
        q = User.objects.filter(email=v)
        if q.count() != 0:
            raise ValueError("Email is not available")
        return v
        
    @validator("password_confirm")
    def passwords_match(cls, v, values, **kwargs):
        password = values.get('password')
        password_confirm = v
        if password != password_confirm:
            raise ValueError("Passwords do not match")
        return v

In [42]:
data = UserSignupSchema(email='abc@teamcfe.com', password='abc123', password_confirm='abc123')

ValidationError: 1 validation error for UserSignupSchema
email
  Email is not available (type=value_error)

In [37]:
data.dict()['password'].get_secret_value()

'abc123'